this notebook creates two maps that show the Hunza watershed with ESRI imagery
- first map overlays the algorithm calculated melt onset dates (MOD) for SIR pixels
- second map overlays the algorithm calculated melt onset dates (MOD) for GRD pixels

the pixel overlays are simply a scatter plot with rectangular markers and are not using EASE-grid projection

## Load in all the modules needed

In [ ]:
%pylab notebook
import matplotlib.pyplot as plt
from netCDF4 import Dataset, num2date
import numpy as np
import pandas as pd
from cetbtools.ease2conv import Ease2Transform
from mpl_toolkits.basemap import Basemap
import glob

In [ ]:
# navigate to where scripts are saved
% cd /mnt/data3/cetb/ipynb_melt_onset/scripts/

In [ ]:
# load the custom functions
from CETB_read_functions import read_Tb
from CETB_read_functions import coords
from CETB_read_functions import calc_DAV
from CETB_read_functions import find_cube_offset
from CETB_read_functions import grid_locations_of_subset
#from CETB_read_functions import find_UIB_cube_offset
#from CETB_read_functions import find_GLaIL_cube_offset
#from CETB_read_functions import grid_locations_of_UIB
#from CETB_read_functions import grid_locations_of_GLaIL
from CETB_algorithms import DAV_MOD
from CETB_analysis import MOD_array
from CETB_analysis import MOD_array_year

## Specify inputs

In [ ]:
#Specify region, satellite, sensor, channel, and image reconstruction algorithm of interest in file name
# this notebook will read in 2 CETB datasets so that channels/algorithms/sensors can be compared
region='GLaIL'  #make this the same syntax as cubefilenames and sub-directory
sat_GRD='F18'   #'AQUA' for AMSRE, 'F13','F14','F15'... for SSMI
sat_SIR= 'F18'
sensor_GRD='SSMIS'  #'AMSRE', 'SSMI', etc.
sensor_SIR='SSMIS'
channel_GRD='37V'  #'36V','36H', '18V','18H', etc. '19V','19H' and '37V','37H' for SSMI)
channel_SIR='37V'
alg_GRD='GRD'   #SIR or GRD
alg_SIR='SIR'

if sensor_GRD=='SSMIS':
    provider='CSU' 
    version='v1.*' #note need to check for different sensors v1.2, 1.3, 1.4
elif sensor_GRD=='AMSRE':
    provider='RSS'
    version='v1.3'

# either change to the directory where the data is or specify it
datadir_GRD = '/mnt/data3/cetb/cubes/'+sat_GRD+'_'+sensor_GRD+'/'+region+'/' 
datadir_SIR = '/mnt/data3/cetb/cubes/'+sat_SIR+'_'+sensor_SIR+'/'+region+'/' 

# prefix filepath
prefix_GRD='CETB.cubefile.'+region+'.'+sat_GRD+'_'+sensor_GRD+'-'+channel_GRD+'-'+alg_GRD+'-'+provider+'-'+version
prefix_SIR='CETB.cubefile.'+region+'.'+sat_SIR+'_'+sensor_SIR+'-'+channel_SIR+'-'+alg_SIR+'-'+provider+'-'+version

# years for each sensor
if sat_GRD=='F13' or sat_SIR=='F13':
    Years = [2002,2003,2004,2005,2006,2007,2008,2009]
elif sat_GRD=='F14' or sat_SIR=='F14':
    Years=[2002,2003,2004,2005,2006,2007,2008]
elif sat_GRD=='F15' or sat_SIR=='F15':
    Years=[2002,2003,2004,2005,2006,2007,2008,2009,2010,2011]
elif sat_GRD=='F18' or sat_SIR=='F18':
    Years=[2010,2011,2012,2013,2014,2015,2016,2017,2018,2019]
elif sat_GRD=='AQUA' or sat_SIR=='AQUA':
    Years=[2003,2004,2005,2006,2007,2008,2009,2010,2011]

#Enter a site name for titles of plots
Site='Vatnajokull, Iceland'

In [ ]:
# SPECIFY latitude and longitude in decimal degrees, need to choose lat/lon corners so that we will load
# in a rectangle of pixels within the corners of these coordinates
lat_start=63.75  #35.9
lat_end=64.88    #37.1
lon_start=-20 #-20 includes katla -18.5vatna    #74
lon_end=-15    #76

In [ ]:
# get cube offset for finding row/col
# function is region specific
find_cube_offset(region,cubeType=None, verbose=False)

In [ ]:
# get the GRD pixel IDs for the lat/lon rectangle chosen
rows_cols_GRD=coords(datadir_GRD, prefix_GRD, lat_start, lat_end, lon_start, lon_end)
rows_cols_GRD

In [ ]:
# row and col numbers of all the SIR pixels contained by the GRD pixels
rows_cols_env=[rows_cols_GRD[0]*8,rows_cols_GRD[1]*8, rows_cols_GRD[2]*8, rows_cols_GRD[3]*8]
rows_cols_env

In [ ]:
#testing syntax of subsetting years
#Years,Years[9:10]

In [ ]:
# load in SIR TB data
subYears=Years
#subYears=Years[9:10]
#data_SIR=read_Tb(datadir_SIR, prefix_SIR, Years,rows_cols_env[0],rows_cols_env[1],rows_cols_env[2],rows_cols_env[3])
CETB_SIR,cal_date,cal_year,cal_month=read_Tb(datadir_SIR, prefix_SIR, subYears,rows_cols_env[0],rows_cols_env[1],rows_cols_env[2],rows_cols_env[3])
#CETB_SIR=data_SIR[0]   # 3-D Tb time-series array of TB
#cal_date=data_SIR[1]    # 1-D array of dates, these will get passed to later functions
#cal_year=data_SIR[2]    # 1-D array of years
#cal_month=data_SIR[3]   # 1-D array of months

# load GRD Tb data
data_GRD=read_Tb(datadir_GRD, prefix_GRD, subYears,rows_cols_GRD[0],rows_cols_GRD[1],rows_cols_GRD[2],rows_cols_GRD[3])
CETB_GRD=data_GRD[0]

In [ ]:
# calculate DAV for the Tb data that was imported
DAV_SIR=calc_DAV(CETB_SIR)
DAV_GRD=calc_DAV(CETB_GRD)

## get the latitudes and longitudes of each pixel for plotting

In [ ]:
#Get lat/lon of each pixel in the subset of 64
# these get used for plotting over basemap
filename=datadir_SIR+prefix_SIR+'.'+str(2019)+'.TB.nc'	
vlist=glob.glob(filename)
data=Dataset(vlist[-1], "r", format="NETCDF4")

lat=data.variables['latitude'] 
lat=lat[:]
lon=data.variables['longitude']
lon=lon[:]
lat_lon=np.dstack((lat,lon))

coordinates=lat_lon[rows_cols_env[0]:rows_cols_env[1],rows_cols_env[2]:rows_cols_env[3]]

coordinates=coordinates.reshape(((rows_cols_env[1]-rows_cols_env[0])*(rows_cols_env[3]-rows_cols_env[2])),2)
pixel_lats=coordinates[:,0]
pixel_lons=coordinates[:,1]

pixel_lats

In [ ]:
#Get lat/lon for GRD pixels
# similar to SIR above
filename=datadir_GRD+prefix_GRD+'.'+str(2019)+'.TB.nc'	
vlist=glob.glob(filename)
data=Dataset(vlist[-1], "r", format="NETCDF4")

lat=data.variables['latitude'] 
lat=lat[:]
lon=data.variables['longitude']
lon=lon[:]
lat_lon=np.dstack((lat,lon))

coordinates=lat_lon[rows_cols_GRD[0]:rows_cols_GRD[1],rows_cols_GRD[2]:rows_cols_GRD[3]]

coordinates=coordinates.reshape(((rows_cols_GRD[1]-rows_cols_GRD[0])*(rows_cols_GRD[3]-rows_cols_GRD[2])),2)
pixel_lats_GRD=coordinates[:,0]
pixel_lons_GRD=coordinates[:,1]

pixel_lats_GRD

In [ ]:
#size of array
shape(pixel_lats),shape(pixel_lats_GRD)

## create arrays of MOD to be used for plotting

In [ ]:
# get array of average MODs for SIR pixels for all the years loaded
#Colorado (Johnson et al 2020) used 5 times in 7 day window
#Patagonia (Monahan and Ramage 2010) and Yukon (Semmens et al 2013?) used 3 times in 5 day window
window=10   # window for MOD algorithm, '10' would be 5 days (2 measurements per day)
count=3    # number of Tb/DAV exceedances to trigger MOD
#From Johnson et al 2020 AMSRE rSIR Tb >= 249 DAV>=13 and AMSRE GRD Tb>=243 DAV>=14
#From Johnson et al 2020 SSMI rSIR and GRD Tb>=247 DAV>=10
DAV_threshold=10
Tb_threshold=247

# sir MOD array - MOD will be in day of year (DOY) #changeback to Years for all years
MOD_DOY_array=MOD_array(datadir_SIR, prefix_SIR, CETB_SIR, DAV_SIR, rows_cols_env, cal_date, subYears, window, count, DAV_threshold, Tb_threshold)
MOD_DOY_array

In [ ]:
# get array of MOD for each pixel SIR - one year of interest
#window=10
#count=3
#DAV_threshold=18
#Tb_threshold=252
year=2019


MOD_DOY_array_year=MOD_array_year(datadir_SIR, prefix_SIR, CETB_SIR, DAV_SIR, rows_cols_env, cal_date, year, window, count, DAV_threshold, Tb_threshold)
MOD_DOY_array_year

In [ ]:
# MOD of the GRD pixel - avg all years
#window=10
#count=3
#DAV_threshold=18
#Tb_threshold=252

MOD_DOY_array_GRD=MOD_array(datadir_GRD, prefix_GRD, CETB_GRD, DAV_GRD, rows_cols_GRD, cal_date, subYears, window, count, DAV_threshold, Tb_threshold)
MOD_DOY_array_GRD

In [ ]:
# get MOD for GRD pixel - one year of interest
#window=10
#count=3
#DAV_threshold=18
#Tb_threshold=252
year=2019
MOD_DOY_array_GRD_year=MOD_array_year(datadir_GRD, prefix_GRD, CETB_GRD, DAV_GRD, rows_cols_GRD, cal_date, year, window, count, DAV_threshold, Tb_threshold)
MOD_DOY_array_GRD_year

## create the maps

In [ ]:
#how to create subplots with basemap (w maps) https://basemaptutorial.readthedocs.io/en/latest/subplots.html

In [ ]:
# PLOT the MOD MAP - avg of all years or one year of interest - SIR
# Choose plot type - 'year' for one year of interest, 'all' for average of all years
plot_type='year'  
#plot_type='all'
year=2019

if plot_type=='all':
    array=MOD_DOY_array
    title=sensor_SIR+'-'+Site+' - MOD - (Avg DOY '+str(Years[0])+'-'+str(Years[-1])+')'
elif plot_type=='year':
    array=MOD_DOY_array_year
    title=sensor_SIR+'-'+Site+' - MOD - ('+str(year)+' DOY)'

fig = plt.figure(figsize=(8,8))
# create figure and axes instances
#fig = plt.figure()
ax = fig.add_axes([0.1,0.1,0.8,0.8])
#ax = fig.add_subplot(121)
# create polar stereographic Basemap instance.
m = Basemap(projection='stere',lon_0=(lon_start+lon_end)/2,lat_0=(lat_start+lat_end)/2,lat_ts=90.,\
            llcrnrlat=(lat_start-.3),urcrnrlat=(lat_end+.5),\
            llcrnrlon=(lon_start-.3),urcrnrlon=(lon_end+.3),\
            rsphere=6371200.,resolution='l',area_thresh=10000, epsg=3857)
m.drawcoastlines()
m.drawcountries()
m.drawstates()
#m.etopo(scale=5, alpha=0.7)
#m.bluemarble(scale=1)
parallels = np.arange(0.,90,0.5)
m.drawparallels(parallels,labels=[1,0,0,0],fontsize=10)
# draw meridians
meridians = np.arange(0,360.,0.5)
m.drawmeridians(meridians,labels=[0,0,0,1],fontsize=10)
m.arcgisimage(service='World_Physical_Map', xpixels = 1500, verbose= True)

#m.readshapefile('/home/mij216/ExploringCETB/IN_Hunza_at_DainyorBridge', 'Hunza', color='red', linewidth=1)
m.readshapefile('/mnt/data3/rgi6.0/06_rgi60_Iceland', 'Iceland', color='red', linewidth=1)

x,y=m(pixel_lons, pixel_lats)
#m.scatter(x,y,c=array, s=30, marker='s',lw=0,cmap='RdBu', alpha=.6) #this line is what was here
m.scatter(x,y,c=array, s=30, marker='s',lw=0,cmap='BuPu_r', alpha=.6) # this line is for testing PuOr, rainbow,binary good for Tb
plt.title(title)

plt.colorbar(label='DOY')
plt.clim(75,175)  #color bar limits

plt.show()


# plot another blank map with no pixel overlay
fig = plt.figure(figsize=(8,8))
ax = fig.add_axes([0.1,0.1,0.8,0.8])

# create polar stereographic Basemap instance.
m = Basemap(projection='stere',lon_0=(lon_start+lon_end)/2,lat_0=(lat_start+lat_end)/2,lat_ts=90.,\
            llcrnrlat=(lat_start-.3),urcrnrlat=(lat_end+.5),\
            llcrnrlon=(lon_start-.3),urcrnrlon=(lon_end+.3),\
            rsphere=6371200.,resolution='l',area_thresh=10000, epsg=3857)
m.drawcoastlines()
m.drawcountries()
m.drawstates()
#m.etopo(scale=5, alpha=0.7)
#m.bluemarble(scale=1)
parallels = np.arange(0.,90,0.5)
m.drawparallels(parallels,labels=[1,0,0,0],fontsize=10)
# draw meridians
meridians = np.arange(180.,360.,0.5)
m.drawmeridians(meridians,labels=[0,0,0,1],fontsize=10)
m.arcgisimage(service='World_Physical_Map', xpixels = 1500, verbose= True)


plt.title(title)
plt.show()


In [ ]:
MOD_DOY_array_GRD

In [ ]:
CETB_SIR

In [ ]:
#color bars here https://matplotlib.org/stable/tutorials/colors/colormaps.html
#documentation on basemap https://matplotlib.org/basemap/users/laea.html

In [ ]:
#np.sqrt((1200)/8)^2
sGRD=700
sSIR=12

In [ ]:
# 2plots left (GRD) and right (SIR)
#how do I set up for year vs average all years?
#redo basemap to be in easegrid projection EASE2.0
fig,axes=plt.subplots(1,2, figsize=(14,5))
#min day of year to plot
minday=75
#max day of year to plot
maxday=175
# create polar stereographic Basemap instance.
axes[0].set_title("MOD GRD")
m = Basemap(ax=axes[0],projection='stere',lon_0=(lon_start+lon_end)/2,lat_0=(lat_start+lat_end)/2,lat_ts=90.,\
            llcrnrlat=(lat_start-.3),urcrnrlat=(lat_end+.5),\
            llcrnrlon=(lon_start-.3),urcrnrlon=(lon_end+.3),\
            rsphere=6371200.,resolution='l',area_thresh=10000, epsg=3857)
m.drawcoastlines()
m.drawcountries()
#m.drawstates()
#m.etopo(scale=3, alpha=0.7)
#m.bluemarble(scale=1)
parallels = np.arange(0.,90,0.5)
m.drawparallels(parallels,labels=[1,0,0,0],fontsize=10)
# draw meridians
meridians = np.arange(0,360.,0.5)
m.drawmeridians(meridians,labels=[0,0,0,1],fontsize=10)
m.arcgisimage(service='World_Physical_Map', xpixels = 1500, verbose= True)

#m.readshapefile('/home/mij216/ExploringCETB/IN_Hunza_at_DainyorBridge', 'Hunza', color='red', linewidth=1)
m.readshapefile('/mnt/data3/rgi6.0/06_rgi60_Iceland', 'Vatnajokull', color='red', linewidth=1)

x,y=m(pixel_lons_GRD, pixel_lats_GRD)
im0=m.scatter(x,y,c=MOD_DOY_array_GRD, s=sGRD, marker='s',lw=0,cmap='BuPu_r', alpha=.6, vmin=minday, vmax=maxday) #RdBu, 
#im0=m.scatter(x,y,c=CETB_SIR, s=1200, marker='s',lw=0,cmap='BuPu_r', alpha=.6) #RdBu, #this didn't work
fig.colorbar(im0, ax=axes[0],label='DOY')
#plt.clim(75,175)  #color bar limits

axes[1].set_title("MOD SIR")
m = Basemap(ax=axes[1],projection='stere',lon_0=(lon_start+lon_end)/2,lat_0=(lat_start+lat_end)/2,lat_ts=90.,\
            llcrnrlat=(lat_start-.3),urcrnrlat=(lat_end+.5),\
            llcrnrlon=(lon_start-.3),urcrnrlon=(lon_end+.3),\
            rsphere=6371200.,resolution='l',area_thresh=10000, epsg=3857)
m.drawcoastlines()
m.drawcountries()
#m.drawstates()
#m.etopo(scale=3, alpha=0.7)
#m.bluemarble(scale=1)
parallels = np.arange(0.,90,0.5)
m.drawparallels(parallels,labels=[1,0,0,0],fontsize=10)
# draw meridians
meridians = np.arange(0,360.,0.5)
m.drawmeridians(meridians,labels=[0,0,0,1],fontsize=10)
m.arcgisimage(service='World_Physical_Map', xpixels = 1500, verbose= True)

#m.readshapefile('/home/mij216/ExploringCETB/IN_Hunza_at_DainyorBridge', 'Hunza', color='red', linewidth=1)
m.readshapefile('/mnt/data3/rgi6.0/06_rgi60_Iceland', 'Vatnajokull', color='red', linewidth=1)

x,y=m(pixel_lons, pixel_lats)
im1=m.scatter(x,y,c=MOD_DOY_array, s=sSIR, marker='s',lw=0,cmap='BuPu_r', alpha=.6,vmin=minday,vmax=maxday) #RdBu, 
fig.colorbar(im1, ax=axes[1],label='DOY')
#plt.clim(75,175)  #color bar limits


plt.show()

In [ ]:
# PLOT the MOD MAP - avg of all years or one year of interest - GRD
# Choose plot type - 'year' for one year of interest, 'all' for average of all years
plot_type='year'  
#plot_type='all'
year=2019

if plot_type=='all':
    array=MOD_DOY_array_GRD
    title=sensor_SIR+'-'+Site+' - MOD - (Avg DOY '+str(Years[0])+'-'+str(Years[-1])+')'
elif plot_type=='year':
    array=MOD_DOY_array_GRD_year
    title=sensor_SIR+'-'+Site+' - MOD - ('+str(year)+' DOY)'

fig = plt.figure(figsize=(8,8))
# create figure and axes instances
#fig = plt.figure()
ax = fig.add_axes([0.1,0.1,0.8,0.8])
#ax = fig.add_subplot(121)
# create polar stereographic Basemap instance.
m = Basemap(projection='stere',lon_0=(lon_start+lon_end)/2,lat_0=(lat_start+lat_end)/2,lat_ts=90.,\
            llcrnrlat=(lat_start-.3),urcrnrlat=(lat_end+.5),\
            llcrnrlon=(lon_start-.3),urcrnrlon=(lon_end+.3),\
            rsphere=6371200.,resolution='l',area_thresh=10000, epsg=3857)
m.drawcoastlines()
m.drawcountries()
#m.drawstates()
#m.etopo(scale=3, alpha=0.7)
#m.bluemarble(scale=1)
parallels = np.arange(0.,90,0.5)
m.drawparallels(parallels,labels=[1,0,0,0],fontsize=10)
# draw meridians
meridians = np.arange(0,360.,0.5)
m.drawmeridians(meridians,labels=[0,0,0,1],fontsize=10)
m.arcgisimage(service='World_Physical_Map', xpixels = 1500, verbose= True)

#m.readshapefile('/home/mij216/ExploringCETB/IN_Hunza_at_DainyorBridge', 'Hunza', color='red', linewidth=1)
m.readshapefile('/mnt/data3/rgi6.0/06_rgi60_Iceland', 'Vatnajokull', color='red', linewidth=1)

x,y=m(pixel_lons_GRD, pixel_lats_GRD)
m.scatter(x,y,c=array, s=1200, marker='s',lw=0,cmap='BuPu_r', alpha=.6) #RdBu, 

plt.title(title)

plt.colorbar(label='DOY')
plt.clim(75,175)  #color bar limits

plt.show()


# plot another blank map with no pixel overlay
fig = plt.figure(figsize=(8,8))
ax = fig.add_axes([0.1,0.1,0.8,0.8])

# create polar stereographic Basemap instance.
m = Basemap(projection='stere',lon_0=(lon_start+lon_end)/2,lat_0=(lat_start+lat_end)/2,lat_ts=90.,\
            llcrnrlat=(lat_start-.3),urcrnrlat=(lat_end+.5),\
            llcrnrlon=(lon_start-.3),urcrnrlon=(lon_end+.3),\
            rsphere=6371200.,resolution='l',area_thresh=10000, epsg=3857)
m.drawcoastlines()
m.drawcountries()
m.drawstates()
#m.etopo(scale=5, alpha=0.7)
#m.bluemarble(scale=1)
parallels = np.arange(0.,90,0.5)
m.drawparallels(parallels,labels=[1,0,0,0],fontsize=10)
# draw meridians
meridians = np.arange(180.,360.,0.5)
m.drawmeridians(meridians,labels=[0,0,0,1],fontsize=10)
m.arcgisimage(service='World_Physical_Map', xpixels = 1500, verbose= True)
#m.readshapefile('/home/mij216/ExploringCETB/IN_Hunza_at_DainyorBridge', 'Hunza', color='red', linewidth=1)
m.readshapefile('/mnt/data3/rgi6.0/06_rgi60_Iceland', 'Vatnajokull', color='red', linewidth=1)

plt.title(title)
plt.show()

In [ ]:
#setup a dataframe so that all the data from ssmi and ssmis are read into one array for long term average
#need to decide which sensor has priority when there is overlap
#can create a reader that loops through enough sensors to get a full time series 
#(skip F10- orbit very elliptical and F19 - short)
#create that time series for the analysis, save for analysis

In [ ]:
#Add to this
#Plot of which pixels have melted as of a specific day

In [ ]:
#figure out refreeze? then from MOD to Freeze, count # of dates with Tb>=threshold (with or without DAV? might depend on site)

In [ ]:
#Add to this
#Plot of how many days (or occurrences) experienced melt 


In [ ]:
#Add to this
#Plot of how many short (1-2 occurrences) melt days EMEs prior to the first MOD
#From Jan 1 - MOD, Number of days with Tb>= threshold and DAV>= threshold (same thresholds)